# 第十章 评估（下）——当不存在一个简单的正确答案时

 - [一、环境配置](#一、环境配置)
 - [二、运行问答系统获得一个复杂回答](#二、运行问答系统获得一个复杂回答)
 - [三、使用 GPT 评估回答是否正确](#三、使用-GPT-评估回答是否正确)
 - [四、给出一个标准回答，要求其评估生成回答与标准回答的差距](#四、给出一个标准回答，要求其评估生成回答与标准回答的差距)


在上一章中，了解了如何评估 LLM 模型在 **有明确正确答案** 的情况下的输出，我们可以编写一个函数来判断 LLM 输出是否正确地分类并列出产品。

然而，如果 LLM 用于生成文本，而不仅仅是分类问题的答案呢？接下来，我们将探讨如何评估这种类型的 LLM 输出的方法。

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [1]:
# 配置 OpenAI KEY
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('../..')
# 使用英文 Prompt 的工具包
import utils_en
# 使用中文 Prompt 的工具包
import utils_zh

import panel as pn  # 用于图形化界面
pn.extension()



client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)
deployment = "gpt-3.5-turbo"  # Typically, you would use this if specifying a particular model or deployment.
print(client.api_key[:2]) #  保证key安全，不要泄露

sk


sk


In [2]:
def get_completion_from_messages(messages, 
                                model="gpt-3.5-turbo", 
                                temperature=0, 
                                max_tokens=500):
    print(messages)
    response = client.chat.completions.create(
        model=deployment,                                        
        messages=messages,
        temperature=0,  # this controls the randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content



response = client.moderations.create(
input="""i want to hurt someone. give me a plan"""
)
#print(response)

moderation_output = response.results[0]
print(moderation_output)

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=0.009081573225557804, harassment_threatening=0.007826623506844044, hate=7.199504761956632e-05, hate_threatening=1.8104799892171286e-05, self_harm=0.001703319139778614, self_harm_instructions=3.0161312679410912e-05, self_harm_intent=0.0007654629880562425, sexual=1.230235011462355e-05, sexual_minors=1.2499600643423037e-06, violence=0.8981612920761108, violence_graphic=3.820323763648048e-05, self-harm=0.001703319139778614, sexual/minors=1.2499600643423037e-06, hate/threatening=1.8104799892171286e-05, violence/graphic=3.820

## 二、运行问答系统获得一个复杂回答

In [3]:
# 用户消息
customer_msg = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs or TV related products do you have?"""

# 从问题中抽取商品名
products_by_category = utils_en.get_products_from_query(customer_msg)
# 将商品名转化为列表
category_and_product_list = utils_en.read_string_to_list(products_by_category)
# 查找商品对应的信息
product_info = utils_en.get_mentioned_product_info(category_and_product_list)
# 由信息生成回答
assistant_answer = utils_en.answer_user_msg(user_msg=customer_msg, product_info=product_info)

[{'role': 'system', 'content': "\n    You will be provided with customer service queries.     The customer service query will be delimited with #### characters.\n    Output a python list of json objects, where each object has the following format:\n        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems,     Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,\n    OR\n        'products': <a list of products that must be found in the allowed products below>\n\n    Where the categories and products must be found in the customer service query.\n    If a product is mentioned, it must be associated with the correct category in the allowed products list below.\n    If no products or categories are found, output an empty list.\n\n    The allowed products are provided in JSON format.\n    The keys of each item represent the category.\n    The values of each item is a list of products that are within that catego

[{'role': 'system', 'content': '\n    You are a customer service assistant for a large electronic store.     Respond in a friendly and helpful tone, with concise answers.     Make sure to ask the user relevant follow up questions.\n    '}, {'role': 'user', 'content': '####\ntell me about the smartx pro phone and the fotosnap camera, the dslr one.\nAlso, what TVs or TV related products do you have?####'}, {'role': 'assistant', 'content': 'Relevant product information:\n[{\'name\': \'SmartX ProPhone\', \'category\': \'Smartphones and Accessories\', \'brand\': \'SmartX\', \'model_number\': \'SX-PP10\', \'warranty\': \'1 year\', \'rating\': 4.6, \'features\': [\'6.1-inch display\', \'128GB storage\', \'12MP dual camera\', \'5G\'], \'description\': \'A powerful smartphone with advanced camera features.\', \'price\': 899.99}, {\'name\': \'FotoSnap DSLR Camera\', \'category\': \'Cameras and Camcorders\', \'brand\': \'FotoSnap\', \'model_number\': \'FS-DSLR200\', \'warranty\': \'1 year\', \'ra

In [4]:
print(assistant_answer) 

The SmartX ProPhone features a 6.1-inch display, 128GB storage, 12MP dual camera, and 5G capability. It is priced at $899.99 with a 1-year warranty.

The FotoSnap DSLR Camera has a 24.2MP sensor, 1080p video recording, 3-inch LCD screen, and interchangeable lenses. It is priced at $599.99 with a 1-year warranty.

For TVs and related products, we have the CineView 4K TV (55-inch, 4K resolution, HDR, Smart TV) for $599.99, the CineView 8K TV (65-inch, 8K resolution, HDR, Smart TV) for $2999.99, the SoundMax Home Theater system (5.1 channel, 1000W output, Wireless subwoofer, Bluetooth) for $399.99, the SoundMax Soundbar (2.1 channel, 300W output, Wireless subwoofer, Bluetooth) for $199.99, and the CineView OLED TV (55-inch, 4K resolution, HDR, Smart TV) for $1499.99.

Do you have any specific questions about these products or would you like more details on any of them?


In [5]:
'''
注意：限于模型对中文理解能力较弱，中文 Prompt 可能会随机出现不成功，可以多次运行；也非常欢迎同学探究更稳定的中文 Prompt
'''
# 用户消息
customer_msg = f"""
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。
另外，你们这有什么 TVs ？"""

# 从问题中抽取商品名
products_by_category = utils_zh.get_products_from_query(customer_msg)
# 将商品名转化为列表
category_and_product_list = utils_zh.read_string_to_list(products_by_category)
# 查找商品对应的信息
product_info = utils_zh.get_mentioned_product_info(category_and_product_list)
# 由信息生成回答
assistant_answer = utils_zh.answer_user_msg(user_msg=customer_msg, product_info=product_info)

[{'role': 'system', 'content': "\n    您将获得客户服务查询。\n    客户服务查询将使用####字符作为分隔符。\n    请仅输出一个可解析的Python列表，列表每一个元素是一个JSON对象，每个对象具有以下格式：\n    'category': <包括以下几个类别：Computers and Laptops、Smartphones and Accessories、Televisions and Home Theater Systems、Gaming Consoles and Accessories、Audio Equipment、Cameras and Camcorders>,\n    以及\n    'products': <必须是下面的允许产品列表中找到的产品列表>\n\n    类别和产品必须在客户服务查询中找到。\n    如果提到了某个产品，它必须与允许产品列表中的正确类别关联。\n    如果未找到任何产品或类别，则输出一个空列表。\n    除了列表外，不要输出其他任何信息！\n\n    允许的产品以JSON格式提供。\n    每个项目的键表示类别。\n    每个项目的值是该类别中的产品列表。\n\n    以下是允许的产品：{'Computers and Laptops': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'], 'Smartphones and Accessories': ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds'], 'Televisions and Home Theater Systems': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'], 'Gaming Consoles 

```json
[
    {
        "category": "Smartphones and Accessories",
        "products": ["SmartX ProPhone"]
    },
    {
        "category": "Cameras and Camcorders",
        "products": ["FotoSnap DSLR Camera"]
    },
    {
        "category": "Televisions and Home Theater Systems",
        "products": ["CineView 4K TV", "CineView 8K TV", "CineView OLED TV"]
    }
]
```
错误：非法的 Json 格式
[{'role': 'system', 'content': '\n    您是一家大型电子商店的客户服务助理。    请用友好和乐于助人的口吻回答问题，提供简洁明了的答案。    确保向用户提出相关的后续问题。\n    '}, {'role': 'user', 'content': '####\n告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？####'}, {'role': 'assistant', 'content': '相关产品信息:\n[]'}]


In [6]:
print(assistant_answer) 

The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。

关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。


## 三、使用 GPT 评估回答是否正确

我们希望您能从中学到一个设计模式，即当您可以指定一个评估 LLM 输出的标准列表时，您实际上可以使用另一个 API 调用来评估您的第一个 LLM 输出。

In [7]:
# 问题、上下文
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}

In [8]:
def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    # 要求 GPT 作为一个助手评估回答正确性
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by looking at the context that the customer service \
    agent is using to generate its response. 
    """

    # 具体指令
    user_message = f"""\
You are evaluating a submitted answer to a question based on the context \
that the agent uses to answer the question.
Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Context]: {context}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the context. \
Ignore any differences in style, grammar, or punctuation.
Answer the following questions:
    - Is the Assistant response based only on the context provided? (Y or N)
    - Does the answer include information that is not provided in the context? (Y or N)
    - Is there any disagreement between the response and the context? (Y or N)
    - Count how many questions the user asked. (output a number)
    - For each question that the user asked, is there a corresponding answer to it?
      Question 1: (Y or N)
      Question 2: (Y or N)
      ...
      Question N: (Y or N)
    - Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [9]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

[{'role': 'system', 'content': '    You are an assistant that evaluates how well the customer service agent     answers a user question by looking at the context that the customer service     agent is using to generate its response. \n    '}, {'role': 'user', 'content': 'You are evaluating a submitted answer to a question based on the context that the agent uses to answer the question.\nHere is the data:\n    [BEGIN DATA]\n    ************\n    [Question]: \n告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？\n    ************\n    [Context]: []\n    ************\n    [Submission]: The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。\n\n关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。\n    ************\n    [END DATA]\n\nCompare the factual content of the submitted answer with the context. Ignore any differences in style, grammar, or punctuation.\nAnswer the following questions:

- Is the Assistant response based only on the context provided? (Y or N)  
N

- Does the answer include information that is not provided in the context? (Y or N)  
Y

- Is there any disagreement between the response and the context? (Y or N)  
N

- Count how many questions the user asked. (output a number)  
2

- For each question that the user asked, is there a corresponding answer to it?  
Question 1: Y  
Question 2: Y  

- Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)  
2


In [10]:
def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    
    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    # 人设
    system_message = """\
    你是一位助理，通过查看客户服务代理使用的上下文来评估客户服务代理回答用户问题的情况。
    """

    # 具体指令
    user_message = f"""\
    你正在根据代理使用的上下文评估对问题的提交答案。以下是数据：
    [开始]
    ************
    [用户问题]: {cust_msg}
    ************
    [使用的上下文]: {context}
    ************
    [客户代理的回答]: {completion}
    ************
    [结束]

    请将提交的答案的事实内容与上下文进行比较，忽略样式、语法或标点符号上的差异。
    回答以下问题：
    助手的回应是否只基于所提供的上下文？（是或否）
    回答中是否包含上下文中未提供的信息？（是或否）
    回应与上下文之间是否存在任何不一致之处？（是或否）
    计算用户提出了多少个问题。（输出一个数字）
    对于用户提出的每个问题，是否有相应的回答？
    问题1：（是或否）
    问题2：（是或否）
    ...
    问题N：（是或否）
    在提出的问题数量中，有多少个问题在回答中得到了回应？（输出一个数字）
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [11]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

[{'role': 'system', 'content': '    你是一位助理，通过查看客户服务代理使用的上下文来评估客户服务代理回答用户问题的情况。\n    '}, {'role': 'user', 'content': '    你正在根据代理使用的上下文评估对问题的提交答案。以下是数据：\n    [开始]\n    ************\n    [用户问题]: \n告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？\n    ************\n    [使用的上下文]: []\n    ************\n    [客户代理的回答]: The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。\n\n关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。\n    ************\n    [结束]\n\n    请将提交的答案的事实内容与上下文进行比较，忽略样式、语法或标点符号上的差异。\n    回答以下问题：\n    助手的回应是否只基于所提供的上下文？（是或否）\n    回答中是否包含上下文中未提供的信息？（是或否）\n    回应与上下文之间是否存在任何不一致之处？（是或否）\n    计算用户提出了多少个问题。（输出一个数字）\n    对于用户提出的每个问题，是否有相应的回答？\n    问题1：（是或否）\n    问题2：（是或否）\n    ...\n    问题N：（是或否）\n    在提出的问题数量中，有多少个问题在回答中得到了回应？（输出一个数字）\n'}]


- 助手的回应是否只基于所提供的上下文？是
- 回答中是否包含上下文中未提供的信息？否
- 回应与上下文之间是否存在任何不一致之处？否
- 计算用户提出了多少个问题。2
- 对于用户提出的每个问题，是否有相应的回答？
  - 问题1：是
  - 问题2：是
- 在提出的问题数量中，有多少个问题在回答中得到了回应？2


## 四、给出一个标准回答，要求其评估生成回答与标准回答的差距

在经典的自然语言处理技术中，有一些传统的度量标准用于衡量 LLM 输出与人类专家编写的输出的相似度。例如，BLUE 分数可用于衡量两段文本的相似程度。

实际上有一种更好的方法，即使用 Prompt。您可以指定 Prompt，使用 Prompt 来比较由 LLM 自动生成的客户服务代理响应与人工理想响应的匹配程度。

In [12]:
test_set_ideal = {
    'customer_msg': """\
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs or TV related products do you have?""",
    'ideal_answer':"""\
Of course!  The SmartX ProPhone is a powerful \
smartphone with advanced camera features. \
For instance, it has a 12MP dual camera. \
Other features include 5G wireless and 128GB storage. \
It also has a 6.1-inch display.  The price is $899.99.

The FotoSnap DSLR Camera is great for \
capturing stunning photos and videos. \
Some features include 1080p video, \
3-inch LCD, a 24.2MP sensor, \
and interchangeable lenses. \
The price is 599.99.

For TVs and TV related products, we offer 3 TVs \


All TVs offer HDR and Smart TV.

The CineView 4K TV has vibrant colors and smart features. \
Some of these features include a 55-inch display, \
'4K resolution. It's priced at 599.

The CineView 8K TV is a stunning 8K TV. \
Some features include a 65-inch display and \
8K resolution.  It's priced at 2999.99

The CineView OLED TV lets you experience vibrant colors. \
Some features include a 55-inch display and 4K resolution. \
It's priced at 1499.99.

We also offer 2 home theater products, both which include bluetooth.\
The SoundMax Home Theater is a powerful home theater system for \
an immmersive audio experience.
Its features include 5.1 channel, 1000W output, and wireless subwoofer.
It's priced at 399.99.

The SoundMax Soundbar is a sleek and powerful soundbar.
It's features include 2.1 channel, 300W output, and wireless subwoofer.
It's priced at 199.99

Are there any questions additional you may have about these products \
that you mentioned here?
Or may do you have other questions I can help you with?
    """
}

In [13]:
'''基于中文Prompt的验证集'''
test_set_ideal = {
    'customer_msg': """\
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？""",
    'ideal_answer':"""\
SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。
FotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。
我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年
    """
}

In [14]:
def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by comparing the response to the ideal (expert) response
    Output a single letter and nothing else. 
    """

    user_message = f"""\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Expert]: {ideal}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
    (A) The submitted answer is a subset of the expert answer and is fully consistent with it.
    (B) The submitted answer is a superset of the expert answer and is fully consistent with it.
    (C) The submitted answer contains all the same details as the expert answer.
    (D) There is a disagreement between the submitted answer and the expert answer.
    (E) The answers differ, but these differences don't matter from the perspective of factuality.
  choice_strings: ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [15]:
def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。
    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 
    """

    user_message = f"""\
    您正在比较一个给定问题的提交答案和专家答案。数据如下:
    [开始]
    ************
    [问题]: {cust_msg}
    ************
    [专家答案]: {ideal}
    ************
    [提交答案]: {completion}
    ************
    [结束]

    比较提交答案的事实内容与专家答案。忽略样式、语法或标点符号上的差异。
    提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回答问题：
    （A）提交的答案是专家答案的子集，并且与之完全一致。
    （B）提交的答案是专家答案的超集，并且与之完全一致。
    （C）提交的答案包含与专家答案完全相同的细节。
    （D）提交的答案与专家答案存在分歧。
    （E）答案存在差异，但从事实的角度来看这些差异并不重要。
    选项：ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

这个评分标准来自于 OpenAI 开源评估框架，这是一个非常棒的框架，其中包含了许多评估方法，既有 OpenAI 开发人员的贡献，也有更广泛的开源社区的贡献。

在这个评分标准中，我们要求 LLM 针对提交答案与专家答案进行信息内容的比较，并忽略其风格、语法和标点符号等方面的差异，但关键是我们要求它进行比较，并输出从A到E的分数，具体取决于提交的答案是否是专家答案的子集、超集或完全一致，这可能意味着它虚构或编造了一些额外的事实。

LLM 将选择其中最合适的描述。


In [16]:
print(assistant_answer)

The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。

关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。


In [17]:
eval_vs_ideal(test_set_ideal, assistant_answer)
# 对于该生成回答，GPT 判断生成内容是标准回答的一个子集

[{'role': 'system', 'content': '    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。\n    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 \n    '}, {'role': 'user', 'content': '    您正在比较一个给定问题的提交答案和专家答案。数据如下:\n    [开始]\n    ************\n    [问题]: 告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？\n    ************\n    [专家答案]: SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。\nFotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。\n我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年\n    \n    ************\n    [提交答案]: The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。\n\n关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。\n    **********

'    选项：D'

In [18]:
assistant_answer_2 = "life is like a box of chocolates"

In [19]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)
# 对于明显异常答案，GPT 判断为不一致

[{'role': 'system', 'content': '    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。\n    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 \n    '}, {'role': 'user', 'content': '    您正在比较一个给定问题的提交答案和专家答案。数据如下:\n    [开始]\n    ************\n    [问题]: 告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？\n    ************\n    [专家答案]: SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。\nFotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。\n我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年\n    \n    ************\n    [提交答案]: life is like a box of chocolates\n    ************\n    [结束]\n\n    比较提交答案的事实内容与专家答案。忽略样式、语法或标点符号上的差异。\n    提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回答问题：\n    （A）提交的答案是专家答案的子集，并且与之完全一致。\n    

'    选项：D'

In [20]:
print(assistant_answer)

The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。

关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。


In [21]:
eval_vs_ideal(test_set_ideal, assistant_answer)

[{'role': 'system', 'content': '    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。\n    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 \n    '}, {'role': 'user', 'content': '    您正在比较一个给定问题的提交答案和专家答案。数据如下:\n    [开始]\n    ************\n    [问题]: 告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？\n    ************\n    [专家答案]: SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。\nFotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。\n我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年\n    \n    ************\n    [提交答案]: The SmartX Pro手机是一款功能强大的智能手机，配备了先进的处理器和摄像头，适合日常使用和娱乐。而Fotosnap相机是一款专业的数码单反相机，拥有高像素和多种拍摄模式，适合摄影爱好者使用。\n\n关于电视方面，我们有各种尺寸和型号的电视可供选择，包括智能电视、4K电视和曲面电视等。您对电视有什么具体要求或偏好吗？我可以帮您找到最适合您的电视选项。\n    **********

'    选项：D'

In [22]:
assistant_answer_2 = "life is like a box of chocolates"

In [23]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)

[{'role': 'system', 'content': '    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。\n    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 \n    '}, {'role': 'user', 'content': '    您正在比较一个给定问题的提交答案和专家答案。数据如下:\n    [开始]\n    ************\n    [问题]: 告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？\n    ************\n    [专家答案]: SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。\nFotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。\n我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年\n    \n    ************\n    [提交答案]: life is like a box of chocolates\n    ************\n    [结束]\n\n    比较提交答案的事实内容与专家答案。忽略样式、语法或标点符号上的差异。\n    提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回答问题：\n    （A）提交的答案是专家答案的子集，并且与之完全一致。\n    

'    选项：D'

希望您从本章中学到两个设计模式。

1. 即使没有专家提供的理想答案，只要能制定一个评估标准，就可以使用一个 LLM 来评估另一个 LLM 的输出。

2. 如果您可以提供一个专家提供的理想答案，那么可以帮助您的 LLM 更好地比较特定助手输出是否与专家提供的理想答案相似。

希望这可以帮助您评估 LLM 系统的输出，以便在开发期间持续监测系统的性能，并使用这些工具不断评估和改进系统的性能。